In [ ]:
import warnings
from pathlib import Path

import iris

In [ ]:
from util_commons import EXPERIMENTS, STASH
from util_mypaths import path_to_processed, path_to_raw

In [ ]:
warnings.filterwarnings("ignore", module="iris")

In [ ]:
# Choose an experiment
exp = "FIRE OLD3"

In [ ]:
# Load .pp
cubelist = iris.load(str(path_to_raw / EXPERIMENTS[exp] / f"{EXPERIMENTS[exp]}_chem.pp"))

Main meteorology:
- air temperature
- air mass
- tropospheric mask

Extra meteorology:
- tropopause altitude

Main gases:
- RH: CH4, C2H6, C3H8
- NOx = NO+NO2
- RONO2: MeONO2, EtONO2, nPrONO2, iPrONO2
- NOy (in addition to NOx and RONO2) = HONO+HO2NO2+HNO3+N2O5+PAN+PPAN
- O3
- OH

Extra gases:
- CO
- HO2
- ROx: MeOO, EtOO, nPrOO, iPrOO
- RCHO (aldehydes): HCHO, MeCHO, EtCHO
- RCOR (ketones): Me2CO

In [ ]:
# Extract, rename and save to .nc
for key, value in STASH["main"].items():
    try:
        cube = cubelist.extract_strict(value)
    except iris.exceptions.ConstraintMismatchError as e:
        print(e)
        if exp in ["BASE OLD", "BASE"] and key == "iprono2":
            # iPrONO2 output was absent from the BASE simulations, but
            # iPrONO2 was chemically inactive in the BASE simulations anyway so use zeroes from the EtONO2 output
            cube = cubelist.extract_strict("m01s34i096")
    cube.rename(key)
    print(cube.name())
    iris.save(cube, str(path_to_processed / EXPERIMENTS[exp] / f"{EXPERIMENTS[exp]}_{key}.nc"))